# Creating an ODE model

In [ ]:

import importlib
from odecell import modelbuilder, solver, paropt
importlib.reload(modelbuilder)
importlib.reload(solver)
importlib.reload(paropt)

from scipy import integrate
from copy import deepcopy
import numpy as np
import pandas as pd
import cobra
from pickle import load, dump
import time, csv

%matplotlib notebook

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

print("\n----------  Building ODE Model ---------------\n")

model = modelbuilder.MetabolicModel()
model.setVerbosity(2)

#metIndx = model.addMetabolite("A_out","Extracellular A", 0.1)
metIndx = model.addMetabolite("A_in","Intracellular A", 0.1)
metIndx = model.addMetabolite("B","Intracellular B", 0.01)
metIndx = model.addMetabolite("C","Intracellular C", 0.02)
metIndx = model.addMetabolite("D","Intracellular D", 0.01)
metIndx = model.addMetabolite("E","Intracellular E", 0.02)
metIndx = model.addMetabolite("F","Intracellular F", 0.02)

rxnIndx = model.addReaction("NewA","zeroOrder","Creation of intracellular A")
model.addParameter(rxnIndx, "K", 0.1, lb=0, ub=0, unit="", parName="")
model.addProduct(rxnIndx,"Prod1","A_in")
#model.getReaction(rxnIndx).setCheckRxn(False) # Overides check for presence of Substrate and Product.

rxnIndx = model.addReaction("AtoB","firstOrder","Conversion of A to B")
model.addSubstrate(rxnIndx, "Sub1", "A_in")
model.addParameter(rxnIndx, "K", 0.1)
model.addProduct(rxnIndx,"Prod1","B")

rxnIndx = model.addReaction("BtoCD","firstOrder","Conversion of B to C and D")
model.addSubstrate(rxnIndx, "Sub1", "B")
model.addParameter(rxnIndx, "K", 0.1)
model.addProduct(rxnIndx,"Prod1","C")
model.addProduct(rxnIndx,"Prod2","D")

rxnIndx = model.addReaction("CDtoE","secondOrder","Conversion of C and D to E")
model.addSubstrate(rxnIndx, "Sub1", "C")
model.addSubstrate(rxnIndx, "Sub2", "D")
model.addParameter(rxnIndx, "K", 0.5, lb=0, ub=10, unit="1/min", parName="CDtoE const")
model.addProduct(rxnIndx,"Prod1","E")

rxnIndx = model.addReaction("DtoF","firstOrder","Conversion of D to F")
model.addSubstrate(rxnIndx, "Sub1", "D")
model.addParameter(rxnIndx, "K", 0.1, lb=0, ub=10, unit="", parName="DtoF const")
model.addProduct(rxnIndx,"Prod1","F")

rxnIndx = model.addReaction("RemoveC","firstOrder","Secretion of intracellular C")
model.addSubstrate(rxnIndx, "Sub1", "C")
model.addParameter(rxnIndx, "K", 0.2, lb=0, ub=10, unit="", parName="Sec C")

rxnIndx = model.addReaction("RemoveE","firstOrder","Secretion of intracellular E")
model.addSubstrate(rxnIndx, "Sub1", "E")
model.addParameter(rxnIndx, "K", 0.1, lb=0, ub=10, unit="", parName="Sec E")

rxnIndx = model.addReaction("RemoveF","firstOrder","Secretion of intracellular F")
model.addSubstrate(rxnIndx, "Sub1", "F")
model.addParameter(rxnIndx, "K", 0.1, lb=0, ub=10, unit="", parName="Sec F")

print("\n----------  Building and Saving COBRA model ---------------\n")

# Creates and saves the model in cobra format.
#cobraModel = model.buildCobraModel("testModel")
#cobra.io.save_json_model(cobraModel,"testModel.json")

print("\n----------  Preparing Solver ---------------\n")

solv = solver.ModelSolver(model)
rxnIdList = solv.buildCall(verbose=2, odeint=True, useJac=True, transpJac=True)

print(rxnIdList)

# Ploting in Escher

Using the cobra model built in the previous step, we can create an Escher model for easy visualization of simualtion results.

In [ ]:
import escher
import escher.urls
import json
import os
from IPython.display import HTML

b = escher.Builder(map_json="toyModel_MAP.json")

b.display_in_notebook(js_source='web', menu='zoom', scroll_behavior='zoom',enable_editing=False)

# Run a short simulation and check the results

In [ ]:
totalTime = 150
step = 0.0001
numbSteps = totalTime/step

tArray = np.linspace(0,totalTime,numbSteps)

print("Running",numbSteps,"ODE steps.\n")

results = integrate.odeint(solv, Dfun=solv.calcJac, y0=model.getInitVals(), \
                                   t=tArray, full_output=False, ixpr=True)

print("Initial concentration: ", model.getInitVals())

print("Final concentration at time " + str(totalTime) + ":", results[-1:])

#print("\nVariation in Metabolite Concentrations:")
#solv(t=0, y=results[len(results)-1,:])

In [ ]:
columnsList = [met.getID() for met in model.getMetList()]

res_df = pd.DataFrame(results, columns=columnsList)
    
t = np.arange(0,len(results),1)
res_df["t"] = t

#res_df.plot(x="t")
res_df.loc[::1000].plot(x="t", ylim=(0,1.2))

In [ ]:
metDataDict = dict()
metDataList = list()

for metID in res_df.columns:
    if metID == "t":
        continue
    #print(metID, float(results_df[results_df.shape[0]-1:][metID]) )
    metDataDict[metID] = float(res_df[res_df.shape[0]-1:][metID])
    metDataList.append(float(res_df[res_df.shape[0]-1:][metID]))

print("\nFinal metabolite concentrations:\n")
for met,val in sorted(metDataDict.items()):
    print(met,":",round(val,3),"mM")
#print(metDataList)

rxnDat = solv.calcFlux(t=0, y=metDataList)

rxnDatDict = {}
for i in range(len(rxnDat)):
    rxnDatDict[ rxnIdList[i] ] = rxnDat[i]

rxnDatDictNorm = rxnDatDict.copy()
for key,val in rxnDatDictNorm.items():
    rxnDatDictNorm[key] = (val/rxnDatDict["NewA"])*100

print("\n---\n")
print("Final normalized reaction fluxes:\n")
for met,val in sorted(rxnDatDictNorm.items()):
    print(met,":",round(val,3))

In [ ]:
import escher
import escher.urls
import json
import os
from IPython.display import HTML

#escher.list_available_maps()
b = escher.Builder(map_json="toyModel_MAP.json",
                   metabolite_data = metDataDict,
                   #reaction_data=rxnDatDict)
                   reaction_data=rxnDatDictNorm)

#reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
#               {'type': 'mean', 'color': '#0000dd', 'size': 20},
#               {'type': 'max', 'color': '#ff0000', 'size': 40}],

b.display_in_notebook(js_source='web', menu='zoom', scroll_behavior='zoom',enable_editing=False)


# Initialize Optimization Object

## Parameters for the optimization object initialization

- Model object
    * desolver object ready to be ran

- Target Group
    * File name containing targets for optimization. Targets can be reaction fluxes or metabolite concentrations. During optimization, model parameters will be changes so that the values produced by the model match the ones provided here.

- Reaction ID for Normalization
    * The flux from this reaction will be used to normalize all other fluxes in the model when comparing them to optimization targets.

- Standardization Method
    * We offer two ways to measure the model's "Distance to Target", the first depends only on the model's output (the "solution") and the target value, while the second depends on a target-specific error value (such as a standard deviation, or another measure of variability).
        * 1) TARGET:   ( (Solution - Target) /Target)**2
        * 2) ERROR:   (Solution - Target)**2/Error

- Loop Time
    * Optimizations are done by repeating small simulation cycles. This argument indicates the total simulated time for each cycle.
    
- dT
    * Time step used in ODE simulation.

- Total Loops
    * Total number of consecutive ODE simulations used to evaluate a proposed parameter set.

- Tolerance
    * Tolerance used when checking for convergence of ODE simulation.

- Use Jacobian
    * Whether the solver object should provide a Jacobian to the ODE integrator used in the optimization.
    
- Use Functor
    * Whether the solver object should build a functor instead of the standard solver interface. (Can speed up the long optimizations but requires the use of Cython)

- Use Cython
    * Whether the solver object should compile the ODE model with Cython (can take many seconds to minutes!)

- Integrator
    * Indicates which Integrator should be used to evolve the ODE model in time. Options are:
        * ode (SciPy)
        * odeint (SciPy)
        * cvodes (pycvodes)




In [ ]:
importlib.reload(paropt)

#loopTime = 15 # in minutes
#dt = 0.0001 # 6 ms
#totalLoops = 20 # 20 iterations of 15 min = 5 hours

opt = paropt.ParOpt(model, trgGrp="", normRxnID="", stdMethod="target", loopTime=15, dt=0.0001, totalLoops=20,
                   tol=0.001, useJac=False, useFunctor=False, useCython=False, integrator="cvodes" )


currParVals = opt.getInitParams()
print(currParVals)

# Set Flux Normalization and Optimization Targets

Normalize by measurement "error":

\begin{equation*}
    O = w_i * \sum_{i=1}^n \left( (v_i - v_i^t)^2 \right) / E_i
\end{equation*}

Normalize by "target" value:

\begin{equation*}
    O = w_i * \sum_{i=1}^n \left( (v_i - v_i^t)/v_i^t \right)^2
\end{equation*}


For testing purposes, we initially set the targets for optimization as the current reaction fluxes. This way, calculating the objective should give us a low "error" or "cost".

In [ ]:
# Sets the Reaction ID for normalization of model fluxes
opt.setNormRxnID("NewA")

# Clear targets (in case this notebook cell is re-executed)
opt.clearTargets()

# Sets optimizations targets
# - targetType: one of [odeRxn | odeMet | fbaRxn]
# - targetID: Reaction ID
# - targetMult: Multiplier, so that different targets can have individual weights
# - targetVal: The target value
# - targetErr: The target variability. (Used with standardization method "error")
# - targetNorm: Indicates whether this target should be normalized
opt.loadTarget("odeRxn", "RemoveE", "1", 53.7, 1, True)
opt.loadTarget("odeRxn", "RemoveF", "1", 46.3, 1, True)

# Calculates one parameter evaluation using the initial parameter values provided with the model.
opt.setIntegrator("cvodes")
opt.calcObjective(currParVals)

Changing the target reaction fluxes slightly and re-calculating the objective function should give us a higher error:

In [ ]:
opt.clearTargets()
opt.loadTarget("odeRxn", "RemoveE", "1", 50, 1, True)
opt.loadTarget("odeRxn", "RemoveF", "1", 50, 1, True)
opt.calcObjective(currParVals)

## Parameter Optimization using Differential Evolution (SciPy)


Now we change the target reaction fluxes considerably, and use a Differential Evolution implementation from SciPy to optimize our parameters, and minimize the error (or cost) using our objective function.

**May take a few of minutes...**

In [ ]:
opt.clearTargets()

# New reaction flux targets.
opt.loadTarget("odeRxn", "RemoveE", "1", 80, 1, True)
opt.loadTarget("odeRxn", "RemoveF", "1", 20, 1, True)

# minimize output from model and solver, since we will run the ODE model several hundred times.
model.setVerbosity(0)
opt.setVerbose(0)

# Optional tracking of optimization evolution.
opt.setOptLogFileName("optimization_log_test.out")

print("\nRunning optimization...\n")
print(time.strftime('%X %x %Z'),"\n")
start_time = time.time()

# Runs the actual differential evolution from SciPy package.
res = opt.optimize(seed = 57321, useJac = True, maxiter=15, strategy="rand1bin")

print("\n","Optimization ended.\n")
print(time.strftime('%X %x %Z'))
elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time/60,"minutes")

print("Final result:", res.x)
pythonResult = res.x

# Plot new results

Now we can re-calculate the ODE model evolution over time to compare with previous behavior.

In [ ]:
model.applyOpt(res.x)

solv = solver.ModelSolver(model)
#solv.buildCall(verbose=1, nocheck=True) # We don't check the model here because we have checked before
solv.buildCall(odeint=True, useJac=True, transpJac=True, verbose=1)

totalTime = 150
step = 0.0001
numbSteps = totalTime/step

tArray = np.linspace(0,totalTime,numbSteps)

print("Running",numbSteps,"ODE steps.")

results,resDict = integrate.odeint(solv, Dfun=solv.calcJac, y0=model.getInitVals(), \
                                   t=tArray, full_output=True, ixpr=True)

print("Initial concentration: ", model.getInitVals())
print("Final concentration at time " + str(totalTime) + ":", results[-1:])

#solv.calc(t=0, y=results[len(results)-1,:])

t = np.arange(0,len(results),1)
res_df = pd.DataFrame(results, columns=[met.getID() for met in model.getMetList()])
res_df["t"] = t

#res_df.plot(x="t")
res_df.loc[::1000].plot(x="t", ylim=(0,1.2))

We can also plot the equilibrium values on our map. Notice the new equilibrium values for reactions "RemoveE" and "RemoveF".

In [ ]:
metDataDict = dict()
metDataList = list()
for metID in res_df.columns:
    #print(metID, float(results_df[results_df.shape[0]-1:][metID]) )
    metDataDict[metID] = float(res_df[res_df.shape[0]-1:][metID])
    metDataList.append(float(res_df[res_df.shape[0]-1:][metID]))

rxnDat = solv.calcFlux(t=0, y=metDataList)

rxnDatDict = {}
for i in range(len(rxnDat)):
    rxnDatDict[ rxnIdList[i] ] = rxnDat[i]

rxnDatDictNorm = rxnDatDict.copy()
for key,val in rxnDatDictNorm.items():
    rxnDatDictNorm[key] = (val/rxnDatDict["NewA"])*100

b = escher.Builder(map_json="toyModel_MAP.json",
                   metabolite_data = metDataDict,
                   reaction_data=rxnDatDictNorm)

b.display_in_notebook(js_source='web', menu='zoom', scroll_behavior='zoom',enable_editing=False)
